# 一些有用的函数

In [3]:
import numpy as np 
import torch 
import torch.nn as nn
import torch.nn.functional as F

In [55]:
## 绝对坐标 -1 到 1
idx = [0, 2]
xx = np.arange(-1, 1.5, 0.5)[idx]
yy = np.arange(-1, 1.5, 0.5)[idx]
v = np.arange(1, 26).reshape(5,5)[None,None]
print(v)
grid = np.meshgrid(*[xx, yy])
grid = [g[:,:,None] for g in grid]
grid = np.concatenate(grid, axis=-1)
grid = grid[None]
grid = torch.from_numpy(grid)

u = torch.nn.functional.grid_sample(torch.from_numpy(v).float(), grid.float(), align_corners=True)
print(u)

[[[[ 1  2  3  4  5]
   [ 6  7  8  9 10]
   [11 12 13 14 15]
   [16 17 18 19 20]
   [21 22 23 24 25]]]]
tensor([[[[ 1.,  3.],
          [11., 13.]]]])


#### scatter_和gather

In [7]:
x = torch.arange(1,9).view(2,4).float()
y = torch.zeros(3,4)
print(x)
print(y)

tensor([[1., 2., 3., 4.],
        [5., 6., 7., 8.]])
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])


In [11]:
## 按dim轴，以给定idx的顺序从x填入y
idx_y = torch.LongTensor([[0,0,0,0],[1,1,1,1]])
z0 = torch.scatter(y, 0, idx_y, x)
print(z0)
idx_y = torch.LongTensor([[0,0,0,0],[2,2,2,2]])
z1 = torch.scatter(y, 0, idx_y, x)
print(z1)

tensor([[1., 2., 3., 4.],
        [5., 6., 7., 8.],
        [0., 0., 0., 0.]])
tensor([[1., 2., 3., 4.],
        [0., 0., 0., 0.],
        [5., 6., 7., 8.]])


In [15]:
idx_y = torch.LongTensor([[0,1,2,3],[0,1,2,3]])
z0 = torch.scatter(y, 1, idx_y, x)
print(z0)
idx_y = torch.LongTensor([[3,2,1,0],[2,2,2,2]])
z1 = torch.scatter(y, 1, idx_y, x)
print(z1)

tensor([[1., 2., 3., 4.],
        [5., 6., 7., 8.],
        [0., 0., 0., 0.]])
tensor([[4., 3., 2., 1.],
        [0., 0., 8., 0.],
        [0., 0., 0., 0.]])


In [23]:
a = torch.FloatTensor(2,4)

In [17]:
from torch.autograd import Variable

In [27]:
a.data.new(2,4).fill_(0.)

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [34]:
y = torch.argmax(x, dim=0)
y

tensor([1, 1, 1, 1])

In [33]:
class_mask = x.data.new(2,4).fill_(0.)
class_mask = Variable(class_mask)
ids = x.view(-1, 4)
class_mask.scatter_(1, torch.LongTensor([[0,]]), 1.)

IndexError: scatter_(): Expected dtype int64 for index.

#### Focal Loss  
$L_{ce} = \begin{cases} -\log \hat{y}, y=1 \\ 
\log (1-\hat{y}), y=0 \end{cases}$  

$L_{fl} = \begin{cases} -(1-\hat{y})^{\gamma} \log \hat{y}, y=1 \\ 
\log (1-\hat{y}), y=0 \end{cases}$

In [47]:
import torch
import torch.nn.functional as F
def sigmoid_focal_loss(x, y, alpha=0.25, gamma=2., reduction='none'):
    """ focal loss. x, y have same shape, y is one-hot encoded """
    p = torch.sigmoid(x) # probability
    p_t = p*y + (1-p) * (1-y) # coefficient
    ce_loss = F.binary_cross_entropy_with_logits(x, y, reduction="none") # cross entropy
    loss = ce_loss * ((1-p_t)** gamma)

    if alpha >= 0:
        alpha_t = alpha*y + (1-alpha) * (1 - y)
        loss = alpha_t * loss

    if reduction == "mean": loss = loss.mean()
    elif reduction == "sum": loss = loss.sum()
    return loss

#### 标签one-hot编码

In [79]:
def to_one_hot1(x, y):
    """ x: result of softmax, y: label """
    N,C,W,H,D = x.shape
    y_one_hot = torch.zeros_like(x)
    m = y.view(-1, 1, W, H, D)
    y_one_hot.scatter_(1, m, 1.)
    return y_one_hot

def to_one_hot2(x, y, num_classes):
    """ x: result of softmax, y: label """
    N,C,W,H,D = x.shape
    y_one_hot = F.one_hot(y.reshape(N,W,H,D), num_classes=num_classes)
    y_one_hot = y_one_hot.permute(0,4,1,2,3).contiguous()
    return y_one_hot

In [4]:
print('ok')

ok


In [3]:
# from manimlib import *

In [4]:
import numpy as np

In [7]:
pqi

NameError: name 'pqi' is not defined

In [ ]:
from manimlib import *

class SquareToCircle(Scene):
    def construct(self):
        circle = Circle()
        circle.set_fill(BLUE, opacity=0.5)
        circle.set_stroke(BLUE_E, width=4)

        self.add(circle)

: 